# Pre-Processing the Dataset
1. Import assessment result <p style="page-break-after: always;"></p>
2. Import building information (geometry + ids) <p style="page-break-after: always;"></p>
3. Then merge them into one dataset <p style="page-break-after: always;"></p>
4. Import simulation features  <p style="page-break-after: always;"></p>

In [1]:
"""STEP 1: load the full feature set"""

import pandas as pd
import numpy as np
import os

# Define the path to your directory
path = 'C:/Users/Home PC/Documents/P5/ProjectCode/Expanded_Experiment/Files/'
os.chdir(path)

Swiss_Sim = pd.read_csv('Features/simulations.csv')

print('simulation data loaded successfully!')

simulation data loaded successfully!


In [2]:
"""STEP 2: Load the assessment results and assign classes"""

# df_2_2 = pd.read_excel('Results/2_2.xlsx', '2_2') 
df_results = pd.read_excel('Results/P5_AssessmentResults.xlsx', 'Sheet1') 
# df_unitID_2_2 = pd.read_excel('Results/unit_id.xlsx', 'unit_id') 
# entity_subtype = pd.read_excel('Results/entity_subtype.xlsx', 'entity_subtype') 
# SeparateTestSet = pd.read_excel('Results/TestSet/2_2_testSet.xlsx', 'test_2_2') 

def create_bins(df, col_name, bin_ranges):
    labels = [label for _, _, label in bin_ranges]
    bins = [lower for lower, _, _ in bin_ranges] + [bin_ranges[-1][1]]  # Include the upper bound of the last range

    df[f'{col_name}_bin'] = pd.cut(df[col_name], bins=bins, labels=labels)
    return df

# Assigning label from performance indicators
bin_ranges = [
    (-0.1, 0.35, '0_insufficient'), 
    (.35, 0.75, '1_sufficient'), 
    (0.75, 1, '2_preferred')]
df_results = create_bins(df_results.copy(), 'LIV_KIT', bin_ranges)

# Add entity_subtype and unit_id in the assessment results df to make it easier to associate with features later on
# df_results = df_results.assign(unit_id=df_unitID_2_2['unit_id'], entity_subtype=df_unitID_2_2['entity_subtype'])



df_results


,LIV_KIT,LIV_BED1,LIV_BED2,LIV_BED3,LIV_BED4,LIV_BED5,LIV_BED6,# OF BEDS,LIV_BATH1,LIV_BATH2,LIV_BATH3,# OF BATH,entity_subtype,unit_id,invalid_geometry,LIV_KIT_bin
0,0.375000,2.250000,0.0000,0.000000,0.0,0.0,0,3,0.000000,0.000000,0.0,2,LIVING,53095,Unit_id 53095: VALID GEOMETRY,1_sufficient
1,0.125000,0.000000,0.0000,0.000000,0.0,0.0,0,3,0.000000,0.000000,0.0,2,LIVING,134183,Unit_id 134183: VALID GEOMETRY,0_insufficient
2,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0,3,0.047619,0.000000,0.0,2,LIVING,134200,Unit_id 134200: VALID GEOMETRY,0_insufficient
3,1.000000,0.062500,0.4375,0.000000,0.0,0.0,0,4,0.000000,0.000000,0.0,2,LIVING,130199,Unit_id 130199: VALID GEOMETRY,2_preferred
4,0.833333,0.000000,0.0000,0.000000,0.0,0.0,0,4,0.000000,0.000000,0.0,2,LIVING,130203,Unit_id 130203: VALID GEOMETRY,2_preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,0.142857,0.285714,0.0000,0.095238,0.0,0.0,0,4,0.142857,0.047619,0.0,2,LIVING,99773,Unit_id 99773: VALID GEOMETRY,0_insufficient
264,0.176471,0.058824,0.0000,0.058824,0.0,0.0,0,3,0.000000,0.058824,0.0,2,LIVING,99760,Unit_id 99760: VALID GEOMETRY,0_insufficient
265,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0,3,0.000000,0.000000,0.0,2,LIVING,100833,Unit_id 100833: VALID GEOMETRY,0_insufficient
266,0.037037,0.074074,0.0000,0.000000,0.0,0.0,0,2,0.037037,0.000000,0.0,1,LIVING,141353,Unit_id 141353: VALID GEOMETRY,0_insufficient


In [3]:
"""STEP 3.1: import building geometry sample"""

geo_268 = pd.read_excel('Geo/top_268_unique_ids.xlsx', 'Top 268 Unique IDs') 
geo_sample = geo_268[geo_268['entity_subtype'] == 'LIVING']
geo_sample = geo_sample.drop_duplicates(subset='unit_id', keep='first')

# drop repeated columns before merging like entity_subtype
df_results = df_results.drop(columns=['entity_subtype'])

geo_sample

,apartment_id,site_id,building_id,plan_id,floor_id,unit_id,area_id,unit_usage,entity_type,entity_subtype,geometry,elevation,height
0,faefdb324eff14ab02268712d80acb7d,85,71,6741,11138,53095,419054.0,RESIDENTIAL,area,LIVING,POLYGON ((-7.5220084330555963 2.16349213609409...,8.7,2.6
114,39e462db543c918e9dd606e0be4bd08c,10965,15885,36002,41112,134183,1418848.0,RESIDENTIAL,area,LIVING,POLYGON ((3.7200228690791732 -4.55284359245765...,0.0,2.6
241,63b72dc183cedd152d68e431e4755912,10965,15885,35990,41122,134200,1418611.0,RESIDENTIAL,area,LIVING,POLYGON ((-0.1435814985152231 0.38055450925636...,2.9,2.6
363,e34971cb892dcecc265250c724afd576,10187,14796,33530,39162,130199,1368775.0,RESIDENTIAL,area,LIVING,POLYGON ((-0.8601146743973374 -7.7752184071551...,8.7,2.6
454,05a9bf439a95a010ec7b988b119aee48,10187,14796,33530,39162,130203,1368766.0,RESIDENTIAL,area,LIVING,POLYGON ((-2.5804317291570378 -0.5262339838532...,8.7,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26698,a0adb931df09f180634a05c56b5f2fde,3889,6076,16586,25424,99773,982081.0,RESIDENTIAL,area,LIVING,POLYGON ((-6.4798081614054226 -18.428409788408...,11.6,2.6
26799,99d0191348ccfe084f5003d0f99291a7,3889,6076,16586,25416,99760,982056.0,RESIDENTIAL,area,LIVING,POLYGON ((-5.8375868424625104 3.42561528013731...,5.8,2.6
26898,a8aaaa5b5090a548fec3998db07e1309,3988,6276,17062,25917,100833,994229.0,RESIDENTIAL,area,LIVING,POLYGON ((2.0276262587675085 -1.58526813705459...,14.5,2.6
26978,c5703544ff2c53cb0ef95a1fd14616a6,11249,16924,38559,44568,141353,1472854.0,RESIDENTIAL,area,LIVING,POLYGON ((-0.6721054614325013 -8.8813485060935...,5.8,2.6


In [4]:
# Check the number of unique 'unit_id' values in the training_set DataFrame
unique_unit_ids = geo_sample['unit_id'].nunique()
print("Number of unique 'unit_id' values:", unique_unit_ids)

Number of unique 'unit_id' values: 268


In [5]:
geo_assessment = pd.merge(geo_sample, df_results, on='unit_id', how='left')

print(geo_assessment.columns)
unique_unit_ids = geo_assessment['unit_id'].nunique()
print("Number of unique 'unit_id' values:", unique_unit_ids)
print(geo_assessment.shape)

Index(['apartment_id', 'site_id', 'building_id', 'plan_id', 'floor_id',
       'unit_id', 'area_id', 'unit_usage', 'entity_type', 'entity_subtype',
       'geometry', 'elevation', 'height', 'LIV_KIT', 'LIV_BED1', 'LIV_BED2',
       'LIV_BED3', 'LIV_BED4', 'LIV_BED5', 'LIV_BED6', '# OF BEDS',
       'LIV_BATH1', 'LIV_BATH2', 'LIV_BATH3', '# OF BATH', 'invalid_geometry',
       'LIV_KIT_bin'],
      dtype='object')
Number of unique 'unit_id' values: 268
(268, 27)


In [6]:
"""STEP 3.2: Map living to kitchen assessment with integer values"""
# Define a mapping dictionary
bin_mapping = {'0_insufficient': 0, '1_sufficient': 1, '2_preferred': 2}

# Map the values and create a new column
geo_assessment['LIV_KIT_bin_mapped'] = geo_assessment['LIV_KIT_bin'].map(bin_mapping)
# print(geo_assessment['LIV_KIT_bin_mapped'])
# print(geo_assessment.shape)

# Remove invalid geometry which are 12 units
geo_assessment = geo_assessment[~geo_assessment['invalid_geometry'].str.contains('INVALID', case=False, na=False)]
geo_assessment



,apartment_id,site_id,building_id,plan_id,floor_id,unit_id,area_id,unit_usage,entity_type,entity_subtype,...,LIV_BED5,LIV_BED6,# OF BEDS,LIV_BATH1,LIV_BATH2,LIV_BATH3,# OF BATH,invalid_geometry,LIV_KIT_bin,LIV_KIT_bin_mapped
0,faefdb324eff14ab02268712d80acb7d,85,71,6741,11138,53095,419054.0,RESIDENTIAL,area,LIVING,...,0.0,0,3,0.000000,0.000000,0.0,2,Unit_id 53095: VALID GEOMETRY,1_sufficient,1
1,39e462db543c918e9dd606e0be4bd08c,10965,15885,36002,41112,134183,1418848.0,RESIDENTIAL,area,LIVING,...,0.0,0,3,0.000000,0.000000,0.0,2,Unit_id 134183: VALID GEOMETRY,0_insufficient,0
2,63b72dc183cedd152d68e431e4755912,10965,15885,35990,41122,134200,1418611.0,RESIDENTIAL,area,LIVING,...,0.0,0,3,0.047619,0.000000,0.0,2,Unit_id 134200: VALID GEOMETRY,0_insufficient,0
3,e34971cb892dcecc265250c724afd576,10187,14796,33530,39162,130199,1368775.0,RESIDENTIAL,area,LIVING,...,0.0,0,4,0.000000,0.000000,0.0,2,Unit_id 130199: VALID GEOMETRY,2_preferred,2
4,05a9bf439a95a010ec7b988b119aee48,10187,14796,33530,39162,130203,1368766.0,RESIDENTIAL,area,LIVING,...,0.0,0,4,0.000000,0.000000,0.0,2,Unit_id 130203: VALID GEOMETRY,2_preferred,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,a0adb931df09f180634a05c56b5f2fde,3889,6076,16586,25424,99773,982081.0,RESIDENTIAL,area,LIVING,...,0.0,0,4,0.142857,0.047619,0.0,2,Unit_id 99773: VALID GEOMETRY,0_insufficient,0
264,99d0191348ccfe084f5003d0f99291a7,3889,6076,16586,25416,99760,982056.0,RESIDENTIAL,area,LIVING,...,0.0,0,3,0.000000,0.058824,0.0,2,Unit_id 99760: VALID GEOMETRY,0_insufficient,0
265,a8aaaa5b5090a548fec3998db07e1309,3988,6276,17062,25917,100833,994229.0,RESIDENTIAL,area,LIVING,...,0.0,0,3,0.000000,0.000000,0.0,2,Unit_id 100833: VALID GEOMETRY,0_insufficient,0
266,c5703544ff2c53cb0ef95a1fd14616a6,11249,16924,38559,44568,141353,1472854.0,RESIDENTIAL,area,LIVING,...,0.0,0,2,0.037037,0.000000,0.0,1,Unit_id 141353: VALID GEOMETRY,0_insufficient,0


CHECKPOINT: visual inspection

In [7]:
"""CHECKPOINT FOR VISUAL INSPECTION"""
# # Specify the filename for the exported Excel file
# export_filename = f'geo_features.xlsx'

# # Specify the worksheet name
# worksheet_name = f'Sheet1'

# # Specify the full path including the filename
# export_path = 'G:/My Drive/06 Academics/TU Delft/Graduation Studio/Design and Digital Files/ProjectCode/Expanded_Experiment/Files/Features/' + export_filename

# # Export the filtered DataFrame to an Excel file with the specified sheet name
# geo_features.to_excel(export_path, index=False, sheet_name=worksheet_name)

# print(export_filename, 'successful', export_path)


'CHECKPOINT FOR VISUAL INSPECTION'

Step 4: Preparing the features

In [8]:
# Create a DataFrame with column indices and names
index_names_df = pd.DataFrame({'Index': range(len(Swiss_Sim.columns)), 'Column Name': Swiss_Sim.columns})

# Convert DataFrame to a list of tuples (index, column name)
index_names_list = list(index_names_df.to_records(index=False))

# Print the list of tuples
print("List of column indices and names:")
print(index_names_list)


List of column indices and names:
[(0, 'site_id'), (1, 'building_id'), (2, 'plan_id'), (3, 'floor_id'), (4, 'unit_id'), (5, 'area_id'), (6, 'unit_usage'), (7, 'apartment_id'), (8, 'layout_compactness'), (9, 'layout_is_navigable'), (10, 'layout_mean_walllengths'), (11, 'layout_area'), (12, 'layout_net_area'), (13, 'layout_room_count'), (14, 'layout_std_walllengths'), (15, 'layout_area_type'), (16, 'layout_number_of_doors'), (17, 'layout_number_of_windows'), (18, 'layout_has_sink'), (19, 'layout_has_shower'), (20, 'layout_has_bathtub'), (21, 'layout_has_stairs'), (22, 'layout_has_entrance_door'), (23, 'layout_has_toilet'), (24, 'layout_perimeter'), (25, 'layout_door_perimeter'), (26, 'layout_window_perimeter'), (27, 'layout_open_perimeter'), (28, 'layout_railing_perimeter'), (29, 'layout_connects_to_bathroom'), (30, 'layout_connects_to_private_outdoor'), (31, 'floor_number'), (32, 'floor_has_elevator'), (33, 'sun_201803210800_max'), (34, 'sun_201803210800_mean'), (35, 'sun_201803210800_m

In [9]:
# STEP 4: Add all fitures from simulation file 
column_indices = [[5,7], # Building information to associate different dataframes with
                  [8,14,24,25,27], # Geometry-based features
                  [16], # Adjacencies and relationships
                  [297,298,299,300,301,302,302], # Centrality 
                  [304,305,306,307,308,309,310], # Distance to entrance door 
                  [311,312,313,314,315,316,317], # Betweeenness
                  [318,319,320,321,322,323,324], # CLosesness
                  [325,326,327,328,329,330,331], # (living?) Room distances
                  [332,333,334,335,336,337,338], # Living - Dining distance
                  [339,340,341,342,343,344,345], # Bathroom distance
                  [346,347,348,349,350,351,352], # Kitchen distance
                  [353,354,355,356,357,358,359], # Distance to balcony
                  [367,368]] # Layout biggest rectangle length and width

Swiss_Sim_Selection = Swiss_Sim.iloc[:, [item for sublist in column_indices for item in sublist]]

# Get column names of Swiss_sim_selected
column_names = list(Swiss_Sim_Selection.columns)
print(column_names)

['area_id', 'apartment_id', 'layout_compactness', 'layout_std_walllengths', 'layout_perimeter', 'layout_door_perimeter', 'layout_open_perimeter', 'layout_number_of_doors', 'connectivity_eigen_centrality_max', 'connectivity_eigen_centrality_mean', 'connectivity_eigen_centrality_median', 'connectivity_eigen_centrality_min', 'connectivity_eigen_centrality_p20', 'connectivity_eigen_centrality_p80', 'connectivity_eigen_centrality_p80', 'connectivity_entrance_door_distance_max', 'connectivity_entrance_door_distance_mean', 'connectivity_entrance_door_distance_median', 'connectivity_entrance_door_distance_min', 'connectivity_entrance_door_distance_p20', 'connectivity_entrance_door_distance_p80', 'connectivity_entrance_door_distance_stddev', 'connectivity_betweenness_centrality_max', 'connectivity_betweenness_centrality_mean', 'connectivity_betweenness_centrality_median', 'connectivity_betweenness_centrality_min', 'connectivity_betweenness_centrality_p20', 'connectivity_betweenness_centrality_p

In [10]:
# Merge geo_features with Swiss_Sim_Selection based on 'area_id' and 'apartment_id'
training_df = pd.merge(geo_assessment, Swiss_Sim_Selection, on=['area_id', 'apartment_id'], how='outer')

# Filter out rows where 'unit_id' from geo_assessment is not present in the merged dataframe
training_df = training_df[training_df['unit_id'].isin(geo_assessment['unit_id'])]

# Check the number of unique 'unit_id' values and the shape of the dataframe
print("Number of unique 'unit_id' values:", geo_assessment['unit_id'].nunique())
print(training_df.shape)



Number of unique 'unit_id' values: 256
(256, 99)


In [11]:
# Remove columns with any NaN values from the DataFrame
training_df = training_df.dropna(axis=1)

# Check the number of unique 'unit_id' values and the shape of the dataframe
print("Number of unique 'unit_id' values:", training_df['unit_id'].nunique())
print(training_df.shape)
print(training_df.columns)

Number of unique 'unit_id' values: 256
(256, 71)
Index(['apartment_id', 'site_id', 'building_id', 'plan_id', 'floor_id',
       'unit_id', 'area_id', 'unit_usage', 'entity_type', 'entity_subtype',
       'geometry', 'elevation', 'height', 'LIV_KIT', 'LIV_BED1', 'LIV_BED2',
       'LIV_BED3', 'LIV_BED4', 'LIV_BED5', 'LIV_BED6', '# OF BEDS',
       'LIV_BATH1', 'LIV_BATH2', 'LIV_BATH3', '# OF BATH', 'invalid_geometry',
       'LIV_KIT_bin', 'LIV_KIT_bin_mapped', 'layout_compactness',
       'layout_std_walllengths', 'layout_perimeter', 'layout_door_perimeter',
       'layout_open_perimeter', 'layout_number_of_doors',
       'connectivity_entrance_door_distance_max',
       'connectivity_entrance_door_distance_mean',
       'connectivity_entrance_door_distance_median',
       'connectivity_entrance_door_distance_min',
       'connectivity_entrance_door_distance_p20',
       'connectivity_entrance_door_distance_p80',
       'connectivity_entrance_door_distance_stddev',
       'connectivity

In [12]:
"""Export training dataset"""

# Specify the filename for the exported Excel file
export_filename = f'training_dataset_P5.xlsx'

# Specify the worksheet name
worksheet_name = f'Sheet1'

# Specify the full path including the filename
export_path = 'G:/My Drive/06 Academics/TU Delft/Graduation Studio/Design and Digital Files/ProjectCode/Expanded_Experiment/Files/Training/' + export_filename

# Export the filtered DataFrame to an Excel file with the specified sheet name
training_df.to_excel(export_path, index=False, sheet_name=worksheet_name)

print(export_filename, 'successful', export_path)